<a href="https://colab.research.google.com/github/JungAh12/Everyone_TF2.0/blob/master/automatic_differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 40kB/s 
     |████████████████████████████████| 501kB 50.3MB/s 
     |████████████████████████████████| 4.3MB 37.0MB/s 


In [2]:
import tensorflow as tf
tf.__version__

'2.0.0-rc1'

#Gradient tape
TF에서 자동 미분을 계산하기 위해 `tf.GradientTape` API를 제공한다.
컨텍스트 안에서 실행된 모든 연산을 테이프에 기록한다.
그다음 reverse mode differentiation을 사용해서 테이프에 기록된 연산의 그래디언트 계산한다.

In [0]:
x = tf.ones([2,2]) #1로 채워진 2x2 매트릭스
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)

#입력 텐서 x에 대한 z의 derivative
dz_dx = t.gradient(z, x)
for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0

In [0]:
x = tf.ones([2,2])

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0

기본적으로 `GradientTape.gradient()`가 호출되면 GradientTape에 포함된 리소스가 해제된다.
동일한 연산에 대해 여러 gradient를 계산하려면 **지속성있는 gradientTape**를 사용하면된다.


In [0]:
x = tf.constant(3.0)    #상수 3.0

with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x*x
    z = y*y
dz_dx = t.gradient(z, x)    # 108.0(4*x^3)
dy_dx = t.gradient(y, x)    # 6.0
del t

#제어 흐름 기록

In [0]:
def f(x, y):
    output = 1.0
    for i in range(y):
        if i > 1 and i< 5:
            output = tf.multiply(output, x)
    return output

def grad(x, y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x, y)
    return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

#Higher-order

In [0]:
x = tf.Variable(1.0)    #1.0으로 초기화된 텐서플로 변수 생성

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x*x*x
    dy_dx = t2.gradient(y, x)   # x에 대해 y 미분
d2y_dx2 = t.gradient(dy_dx, x)  # x에 대해 한 번 더 dy_dx 미분

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0

In [14]:
dy_dx

<tf.Tensor: id=167, shape=(), dtype=float32, numpy=3.0>

In [15]:
d2y_dx2

<tf.Tensor: id=182, shape=(), dtype=float32, numpy=6.0>